# [AICE Professional 문제풀이 특강] (Tabular) 통신사 이탈 여부 예측
+ AICE Professioanl Tabular 문제 입니다.
+ 데이터 : 통신사 이탈 데이터
+ 목표 : 해당 데이터를 가지고 통신사 이탈 여부 예측하는 최고의 성능의 모델을 만들기!!
  - 달성목표 : Accuracy 77% 이상


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, Flatten
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
import pandas as pd

df = pd.read_csv('https://github.com/gzone1357/Image1/raw/main/telco_churn_data_v1.csv')
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1090-PYKCI,Female,0.0,Yes,Yes,69,Yes,Yes,Fiber optic,Yes,...,Yes,Yes,Yes,No,One year,Yes,Credit card (automatic),105.10,7234.8,No
1,5835-BEQEU,Male,0.0,No,No,3,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,76.95,228.4,Yes
2,9943-VSZUV,Male,1.0,No,No,67,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,75.70,5060.85,No


In [ ]:
df = df.drop('customerID', axis=1)

In [ ]:
df.isnull().sum()

,0
gender,9
SeniorCitizen,1
Partner,0
Dependents,2
tenure,0
PhoneService,3
MultipleLines,0
InternetService,0
OnlineSecurity,0
OnlineBackup,0


In [ ]:
df = df.drop('DeviceProtection', axis=1)

In [ ]:
df.isnull().sum()

,0
gender,9
SeniorCitizen,1
Partner,0
Dependents,2
tenure,0
PhoneService,3
MultipleLines,0
InternetService,0
OnlineSecurity,0
OnlineBackup,0


In [ ]:
df = df.dropna()

In [ ]:
df

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0.0,Yes,Yes,69,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,No,One year,Yes,Credit card (automatic),105.10,7234.8,No
1,Male,0.0,No,No,3,Yes,Yes,Fiber optic,No,No,No,No,No,Month-to-month,Yes,Electronic check,76.95,228.4,Yes
2,Male,1.0,No,No,67,Yes,Yes,Fiber optic,No,No,No,No,No,Month-to-month,Yes,Mailed check,75.70,5060.85,No
3,Male,1.0,Yes,No,16,Yes,No,DSL,Yes,Yes,No,No,No,Month-to-month,Yes,Bank transfer (automatic),54.55,825.1,No
4,Male,0.0,Yes,No,43,Yes,No,DSL,No,No,No,Yes,No,One year,Yes,Mailed check,56.35,2391.15,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Female,0.0,No,No,4,Yes,No,Fiber optic,No,No,No,No,Yes,Month-to-month,Yes,Credit card (automatic),82.85,319.6,No
7039,Male,0.0,Yes,Yes,25,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),25.50,630.6,No
7040,Male,0.0,Yes,No,3,Yes,No,DSL,No,No,Yes,No,No,Month-to-month,Yes,Mailed check,55.80,154.55,No
7041,Male,0.0,Yes,Yes,4,Yes,No,DSL,No,No,Yes,No,Yes,Month-to-month,No,Credit card (automatic),61.45,229.55,Yes


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7027 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7027 non-null   object 
 1   SeniorCitizen     7027 non-null   float64
 2   Partner           7027 non-null   object 
 3   Dependents        7027 non-null   object 
 4   tenure            7027 non-null   int64  
 5   PhoneService      7027 non-null   object 
 6   MultipleLines     7027 non-null   object 
 7   InternetService   7027 non-null   object 
 8   OnlineSecurity    7027 non-null   object 
 9   OnlineBackup      7027 non-null   object 
 10  TechSupport       7027 non-null   object 
 11  StreamingTV       7027 non-null   object 
 12  StreamingMovies   7027 non-null   object 
 13  Contract          7027 non-null   object 
 14  PaperlessBilling  7027 non-null   object 
 15  PaymentMethod     7027 non-null   object 
 16  MonthlyCharges    7027 non-null   float64
 17  

In [ ]:
df = df[df['TotalCharges'] !=' ']

In [ ]:
df['TotalCharges'] = df['TotalCharges'].astype('float')

<ipython-input-26-cadf4347bbaa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TotalCharges'] = df['TotalCharges'].astype('float')


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7016 entries, 0 to 7042
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7016 non-null   object 
 1   SeniorCitizen     7016 non-null   float64
 2   Partner           7016 non-null   object 
 3   Dependents        7016 non-null   object 
 4   tenure            7016 non-null   int64  
 5   PhoneService      7016 non-null   object 
 6   MultipleLines     7016 non-null   object 
 7   InternetService   7016 non-null   object 
 8   OnlineSecurity    7016 non-null   object 
 9   OnlineBackup      7016 non-null   object 
 10  TechSupport       7016 non-null   object 
 11  StreamingTV       7016 non-null   object 
 12  StreamingMovies   7016 non-null   object 
 13  Contract          7016 non-null   object 
 14  PaperlessBilling  7016 non-null   object 
 15  PaymentMethod     7016 non-null   object 
 16  MonthlyCharges    7016 non-null   float64
 17  

In [ ]:
obj_col = df.select_dtypes('object').columns
obj_col

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'Churn'],
      dtype='object')

In [ ]:
df = pd.get_dummies(data=df, columns=obj_col, drop_first=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7016 entries, 0 to 7042
Data columns (total 29 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7016 non-null   float64
 1   tenure                                 7016 non-null   int64  
 2   MonthlyCharges                         7016 non-null   float64
 3   TotalCharges                           7016 non-null   float64
 4   gender_Male                            7016 non-null   bool   
 5   Partner_Yes                            7016 non-null   bool   
 6   Dependents_Yes                         7016 non-null   bool   
 7   PhoneService_Yes                       7016 non-null   bool   
 8   MultipleLines_No phone service         7016 non-null   bool   
 9   MultipleLines_Yes                      7016 non-null   bool   
 10  InternetService_Fiber optic            7016 non-null   bool   
 11  InternetS

In [ ]:
features = df.drop('Churn_Yes', axis=1)
labels = df['Churn_Yes']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(features, labels, test_size=0.2, random_state=42, stratify=labels)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((5612, 28), (1404, 28), (5612,), (1404,))

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
lgr = LogisticRegression()
lgr.fit(x_train, y_train)
lgr.score(x_valid, y_valid)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7991452991452992

In [ ]:
sgdc = SGDClassifier()
sgdc.fit(x_train, y_train)
sgdc.score(x_valid, y_valid)

0.7770655270655271

In [ ]:
labels.value_counts()

,count
Churn_Yes,
False,5150
True,1866


In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(x_train.shape[-1], )))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1)
mc = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
history = model.fit(
    x_train, y_train,
    epochs=10,
    validation_data=(x_valid, y_valid),
    callbacks=[es, mc],
    verbose=1
)

Epoch 1/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6475 - loss: 0.7367
Epoch 1: val_loss improved from inf to 0.56073, saving model to best_model.keras
176/176 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 0.6477 - loss: 0.7363 - val_accuracy: 0.7707 - val_loss: 0.5607
Epoch 2/10
173/176 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7445 - loss: 0.5895
Epoch 2: val_loss did not improve from 0.56073
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7445 - loss: 0.5894 - val_accuracy: 0.7635 - val_loss: 0.5635
Epoch 3/10
161/176 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7455 - loss: 0.5607
Epoch 3: val_loss did not improve from 0.56073
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7449 - loss: 0.5611 - val_accuracy: 0.5114 - val_loss: 0.7404
Epoch 4/10
165/176 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7514 - loss: 0.5515
Epoch 4: val_loss improved from 0.56073 to 0.51837, saving model to best_model.keras
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - 

In [ ]:
model_result = model.predict(x_valid)
y_pred = np.argmax(model_result, axis=1)
y_pred

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
lgr_y_pred = lgr.predict(x_valid)
lgr_y_pred

array([False, False, False, ..., False, False,  True])

In [ ]:
df5 = pd.DataFrame()
df5['label'] = y_valid
df5['pred'] = lgr_y_pred
df5

,label,pred
1478,False,False
6610,False,False
2059,True,False
5314,False,False
5414,False,False
...,...,...
1125,False,False
5737,False,True
844,False,False
5263,False,False


In [ ]:
df5['pred'].value_counts()

,count
pred,
False,1101
True,303


In [ ]:
df5.to_csv('01043620081_1.csv', index=False, encoding='utf-8')
df6 = pd.read_csv('01043620081_1.csv')
df6

,label,pred
0,False,False
1,False,False
2,True,False
3,False,False
4,False,False
...,...,...
1399,False,False
1400,False,True
1401,False,False
1402,False,False


In [ ]:
import joblib
joblib.dump(lgr, '01043620081_1.joblib')

['01043620081_1.joblib']

# AICE Professional 특강 - 텍스트 실습
+ 본과정에서는 AI-HUB 감정말뭉치 데이터를 가지고 텍스트 언어 모델을 만들고
 감성분류 실습하겠습니다.
+ 첫번째는 감정말뭉치 데이터를 전처리하고, TF-IDF 토큰나이져를 활용하여 토큰화하고 머신러닝으로 모델링 만들고 감성분류 수행합니다.
+ 두번째는 keras 토큰나이져를 활용하여 토큰화하고 일정 크기의 문장으로 맞추기 위해 패딩 및 embedding 레이어를 이용해서 단어를 n차원 밀집벡터를 만들고 train시에 학습되도록 합니다.그리고 나서 LSTM 모델을 이용하여 감성분류를 수행합니다.


In [ ]:
# AI-HUB 감성 대화 말뭉치 활용하여 만든 데이터 읽어오기
text_df = pd.read_csv ('https://github.com/ohgzone/file1/raw/main/aihub_coupus.csv' )
text_df.head(3)

,감정,문장
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...


In [ ]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51630 entries, 0 to 51629
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   감정      51630 non-null  object
 1   문장      51630 non-null  object
dtypes: object(2)
memory usage: 806.8+ KB


In [ ]:
text_df[text_df['문장'].str.contains('[^가-힣 ]')].head(3)

,감정,문장
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...


In [ ]:
text_df['문장'] = text_df['문장'].str.replace('[^가-힣 ]', '', regex=True)
text_df['문장'] = text_df['문장'].str.strip()

In [ ]:
text_df['문장'].duplicated().sum()

56

In [ ]:
text_df.drop_duplicates(subset='문장', inplace=True)

In [ ]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51574 entries, 0 to 51629
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   감정      51574 non-null  object
 1   문장      51574 non-null  object
dtypes: object(2)
memory usage: 1.2+ MB


In [ ]:
text_class_to_label = {'불안':0, "분노":1, '상처':2, '슬픔':3, '당황':4, '기쁨':5}

In [ ]:
text_df['감정'].value_counts()

,count
감정,
불안,9311
분노,9149
상처,9130
슬픔,9118
당황,8747
기쁨,6119


In [ ]:
text_df['감정'] = text_df['감정'].map(text_class_to_label)
text_df['감정'].value_counts()

,count
감정,
0,9311
1,9149
2,9130
3,9118
4,8747
5,6119


In [ ]:
text_features = text_df['문장']
text_labels = text_df['감정']

In [ ]:
text_x_train, text_x_valid, text_y_train, text_y_valid = train_test_split(text_features, text_labels, test_size=0.2, random_state=42, stratify=text_labels)
text_x_train.shape, text_x_valid.shape, text_y_train.shape, text_y_valid.shape

((41259,), (10315,), (41259,), (10315,))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
text_x_train_v = tfidf.fit_transform(text_x_train)
text_x_valid_v = tfidf.transform(text_x_valid)

In [ ]:
text_lgr = LogisticRegression()
text_lgr.fit(text_x_train_v, text_y_train)
text_lgr.score(text_x_valid_v, text_y_valid)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5071255453223461

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_x_train)

In [ ]:
print(tokenizer.word_index)

{'너무': 1, '내가': 2, '내': 3, '것': 4, '나': 5, '같아': 6, '안': 7, '나는': 8, '요즘': 9, '친구가': 10, '못': 11, '나를': 12, '때문에': 13, '정말': 14, '오늘': 15, '게': 16, '수': 17, '잘': 18, '우리': 19, '있어': 20, '많이': 21, '없어': 22, '슬퍼': 23, '힘들어': 24, '해서': 25, '들어': 26, '난': 27, '다': 28, '하고': 29, '않아': 30, '화가': 31, '봐': 32, '더': 33, '할': 34, '이제': 35, '있는데': 36, '하는': 37, '있는': 38, '자꾸': 39, '아내가': 40, '남편이': 41, '해': 42, '한': 43, '거': 44, '왜': 45, '일을': 46, '같아서': 47, '하는데': 48, '없어서': 49, '했는데': 50, '모르겠어': 51, '했어': 52, '어떻게': 53, '일이': 54, '나이가': 55, '다른': 56, '친구들이': 57, '마음이': 58, '날': 59, '회사에서': 60, '이번에': 61, '불안해': 62, '걸': 63, '돈을': 64, '돼': 65, '속상해': 66, '해야': 67, '좋은': 68, '기분이': 69, '걱정이야': 70, '있어서': 71, '이': 72, '부모님이': 73, '말을': 74, '때': 75, '혼자': 76, '좋아': 77, '그': 78, '나만': 79, '몸이': 80, '싶어': 81, '아들이': 82, '할지': 83, '나도': 84, '갑자기': 85, '계속': 86, '보니': 87, '어제': 88, '같이': 89, '없는': 90, '돈이': 91, '엄마가': 92, '열심히': 93, '이번': 94, '않아서': 95, '사람이': 96, '이렇게': 97, '나한테': 98, '회사': 99, '지금':

In [ ]:
print(tokenizer.word_counts)

OrderedDict([('동료들의', 5), ('추천으로', 5), ('내가', 4788), ('이달의', 2), ('사원상을', 1), ('받게', 45), ('됐어', 339), ('최근에', 122), ('성적이', 460), ('떨어지고', 39), ('있어서', 612), ('스트레스', 140), ('받아', 225), ('아빠는', 113), ('일도', 144), ('안', 3251), ('하고', 1108), ('매일', 370), ('술만', 28), ('먹어', 31), ('진저리가', 2), ('나', 3597), ('질환', 16), ('때문에', 1773), ('병원', 75), ('가까운', 20), ('곳으로', 30), ('이사를', 35), ('오다', 1), ('보니', 530), ('아는', 98), ('사람이', 509), ('없어서', 813), ('외로워', 266), ('식사하는데', 2), ('맞은', 15), ('편에', 1), ('앉은', 6), ('자꾸', 895), ('나를', 1794), ('쳐다보는', 19), ('것', 3759), ('같아서', 843), ('신경', 162), ('쓰였어', 2), ('조건만', 2), ('보고', 361), ('결혼하는', 21), ('것은', 55), ('너무', 8348), ('한심해', 251), ('직장생활도', 3), ('재미없고', 5), ('병이', 116), ('있으니까', 24), ('무언가를', 13), ('하자고', 29), ('마음먹는', 1), ('게', 1632), ('어려워', 216), ('이제', 946), ('다', 1140), ('늙어가는데', 2), ('몸이', 576), ('아파', 232), ('건강하게', 53), ('늙지', 1), ('못하는', 131), ('자신이', 436), ('짜증', 398), ('의사가', 111), ('나는', 2094), ('채식주의를', 1), ('고수해서', 1), ('악성', 3), (

In [ ]:
max_words = len(tokenizer.word_index)
max_words

47632

In [ ]:
text_x_train_seq = tokenizer.texts_to_sequences(text_x_train)
text_x_valid_seq = tokenizer.texts_to_sequences(text_x_valid)

In [ ]:
max(len(i) for i in text_x_train_seq)

39

In [ ]:
text_x_train_pad = pad_sequences(text_x_train_seq, maxlen=39)
text_x_valid_pad = pad_sequences(text_x_valid_seq, maxlen=39)

In [ ]:
text_model = Sequential()
text_model.add(Embedding(input_dim=max_words+1, output_dim=100, input_length=39))
text_model.add(Bidirectional(LSTM(16, return_sequences=True)))
text_model.add(Bidirectional(LSTM(16, return_sequences=False)))
text_model.add(Dense(64, activation='swish'))
text_model.add(BatchNormalization())
text_model.add(Dropout(0.5))
text_model.add(Dense(6, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
text_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
es2 = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=1)
mc2 = ModelCheckpoint('best_text_model.keras', monitor='val_loss', save_best_only=True, verbose=1)

history2 = text_model.fit(
    text_x_train_pad, text_y_train,
    epochs=3,
    batch_size=512,
    validation_data=(text_x_valid_pad, text_y_valid),
    callbacks=[es2, mc2],
    verbose=1
)

Epoch 1/3
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3048 - loss: 1.6641
Epoch 1: val_loss improved from inf to 1.73189, saving model to best_text_model.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 15s 46ms/step - accuracy: 0.3059 - loss: 1.6624 - val_accuracy: 0.2771 - val_loss: 1.7319
Epoch 2/3
80/81 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6407 - loss: 1.0294
Epoch 2: val_loss improved from 1.73189 to 1.67288, saving model to best_text_model.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.6406 - loss: 1.0291 - val_accuracy: 0.3921 - val_loss: 1.6729
Epoch 3/3
81/81 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8115 - loss: 0.5878
Epoch 3: val_loss improved from 1.67288 to 1.60990, saving model to best_text_model.keras
81/81 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.8114 - loss: 0.5880 - val_accuracy: 0.3388 - val_loss: 1.6099
Restoring model weights from the end of the best epoch: 3.


In [ ]:
text_lgr_y_pred = text_lgr.predict(text_x_valid_v)

In [ ]:
text_df5 = pd.DataFrame()
text_df5['label'] = text_y_valid
text_df5['pred'] = text_lgr_y_pred
text_df5

,label,pred
1664,3,3
46890,3,3
41192,0,2
17562,0,0
20251,3,3
...,...,...
35357,3,3
41726,2,1
45104,0,0
14418,1,1


In [ ]:
text_label_to_class = {0:'불안', 1:"분노", 2:'상처', 3:'슬픔', 4:'당황', 5:'기쁨'}
text_df5['label'] = text_df5['label'].map(text_label_to_class)
text_df5['pred'] = text_df5['pred'].map(text_label_to_class)

In [ ]:
text_df5

,label,pred
1664,슬픔,슬픔
46890,슬픔,슬픔
41192,불안,상처
17562,불안,불안
20251,슬픔,슬픔
...,...,...
35357,슬픔,슬픔
41726,상처,분노
45104,불안,불안
14418,분노,분노


In [ ]:
text_df5.to_csv('01043620081_2.csv', index=False, encoding='utf-8')
text_df6 = pd.read_csv('01043620081_2.csv')
text_df6

,label,pred
0,슬픔,슬픔
1,슬픔,슬픔
2,불안,상처
3,불안,불안
4,슬픔,슬픔
...,...,...
10310,슬픔,슬픔
10311,상처,분노
10312,불안,불안
10313,분노,분노


In [ ]:
joblib.dump(text_lgr, '01043620081_2.joblib')

['01043620081_2.joblib']

# AICE Professional 특강 - 이미지 실습
+ 본과정에서는 꽃 이미지 데이터를 가지고 이미지 모델을 만들고
 분류 실습하겠습니다.
+ 첫번째로, 이미지 파일 읽고 , 라벨정보도 만들어 데이터셋을 생성하고
+ 만들어진 데이터셋을 활용하여 CNN 모델 학습하고 평가하겠습니다.
+ 두번째로, 이미지 파일 읽고, 라벨정보 만드는 작업을 Keras의 image_dataset_from_directory 함수 활용하여
+ 데이터셋을 만들고 CNN 모델 학습 및 평가 하겠습니다.
+ 세번째로, MobileNet Transfer Learning(전이학습)으로 모델 학습 및 평가 진행하겠습니다.

In [ ]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)

228813984/228813984 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
!ls -l /root/.keras/datasets/flower_photos/flower_photos

total 612
drwx------ 2 270850 5000  36864 Feb 10  2016 daisy
drwx------ 2 270850 5000  49152 Feb 10  2016 dandelion
-rw-r----- 1 270850 5000 418049 Feb  9  2016 LICENSE.txt
drwx------ 2 270850 5000  36864 Feb 10  2016 roses
drwx------ 2 270850 5000  36864 Feb 10  2016 sunflowers
drwx------ 2 270850 5000  45056 Feb 10  2016 tulips


In [ ]:
!ls -l /root/.keras/datasets/flower_photos/flower_photos/daisy | grep jpg | wc -l
!ls -l /root/.keras/datasets/flower_photos/flower_photos/daisy | grep jpeg | wc -l
!ls -l /root/.keras/datasets/flower_photos/flower_photos/daisy | grep png | wc -l
!ls -l /root/.keras/datasets/flower_photos/flower_photos/daisy | grep gif | wc -l

633
0
0
0


In [ ]:
train_img_path = '/root/.keras/datasets/flower_photos/flower_photos/'

In [ ]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_img_path,
    labels='inferred',
    #class_names=['cats', 'dogs']
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=(224,224),
    seed=42,
    shuffle=True,
    validation_split=0.2,
    subset='training'
)
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=train_img_path,
    labels='inferred',
    #class_names=['cats', 'dogs']
    label_mode='categorical',
    color_mode='rgb',
    batch_size=32,
    image_size=(224,224),
    seed=42,
    shuffle=True,
    validation_split=0.2,
    subset='validation'
)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [ ]:
img_base_model = tf.keras.applications.MobileNetV3Large(input_shape=(224,224,3), weights='imagenet', include_top=False)
img_base_model.trainable=False

12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
len(train_dataset.class_names)

5

In [ ]:
img_model = Sequential()
img_model.add(img_base_model)
img_model.add(Flatten())
img_model.add(Dense(256, activation=None))
img_model.add(BatchNormalization())
img_model.add(Activation('relu'))
img_model.add(Dropout(0.5))
img_model.add(Dense(5, activation='softmax'))
img_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ MobileNetV3Large (Functional)        │ (None, 7, 7, 960)           │       2,996,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 47040)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 256)                 │      12,042,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_20 (Activation)           │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 5)                   │           1,285 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,041,157 (57.38 MB)

 Trainable params: 12,044,293 (45.95 MB)

 Non-trainable params: 2,996,864 (11.43 MB)

In [ ]:
img_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
es3 = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True, verbose=1)
mc3 = ModelCheckpoint('best_img_model.keras', monitor='val_loss', save_best_only=True, verbose=1)
history3 = img_model.fit(
    train_dataset,
    epochs=3,
    validation_data=valid_dataset,
    callbacks=[es3, mc3],
    verbose=1
)

Epoch 1/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.7299 - loss: 0.7451
Epoch 1: val_loss improved from inf to 0.43909, saving model to best_img_model.keras
92/92 ━━━━━━━━━━━━━━━━━━━━ 50s 320ms/step - accuracy: 0.7308 - loss: 0.7427 - val_accuracy: 0.8787 - val_loss: 0.4391
Epoch 2/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.9662 - loss: 0.1226
Epoch 2: val_loss improved from 0.43909 to 0.33954, saving model to best_img_model.keras
92/92 ━━━━━━━━━━━━━━━━━━━━ 10s 109ms/step - accuracy: 0.9663 - loss: 0.1223 - val_accuracy: 0.8924 - val_loss: 0.3395
Epoch 3/3
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.9907 - loss: 0.0550
Epoch 3: val_loss did not improve from 0.33954
92/92 ━━━━━━━━━━━━━━━━━━━━ 8s 89ms/step - accuracy: 0.9907 - loss: 0.0549 - val_accuracy: 0.8965 - val_loss: 0.3430
Restoring model weights from the end of the best epoch: 2.


In [ ]:
img_glob = sorted(glob.glob('/root/.keras/datasets/flower_photos/flower_photos/*/*jpg'))
img_file = [os.path.basename(path) for path in img_glob]
img_label = [os.path.basename(os.path.dirname(path)) for path in img_glob]

img_df5 = pd.DataFrame()
img_df5['image'] = img_glob
img_df5['file'] = img_file
img_df5['label'] = img_label
img_df5

,image,file,label
0,/root/.keras/datasets/flower_photos/flower_pho...,100080576_f52e8ee070_n.jpg,daisy
1,/root/.keras/datasets/flower_photos/flower_pho...,10140303196_b88d3d6cec.jpg,daisy
2,/root/.keras/datasets/flower_photos/flower_pho...,10172379554_b296050f82_n.jpg,daisy
3,/root/.keras/datasets/flower_photos/flower_pho...,10172567486_2748826a8b.jpg,daisy
4,/root/.keras/datasets/flower_photos/flower_pho...,10172636503_21bededa75_n.jpg,daisy
...,...,...,...
3665,/root/.keras/datasets/flower_photos/flower_pho...,9831362123_5aac525a99_n.jpg,tulips
3666,/root/.keras/datasets/flower_photos/flower_pho...,9870557734_88eb3b9e3b_n.jpg,tulips
3667,/root/.keras/datasets/flower_photos/flower_pho...,9947374414_fdf1d0861c_n.jpg,tulips
3668,/root/.keras/datasets/flower_photos/flower_pho...,9947385346_3a8cacea02_n.jpg,tulips


In [ ]:
train_dataset.class_names

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [ ]:
img_predictions = []
img_folder = '/root/.keras/datasets/flower_photos/flower_photos/*/'
img_label_to_class = {
    0:'daisy',
    1:'dandelion',
    2:'roses',
    3:'sunflowers',
    4:'tulips'
}

for i, row in img_df5.iterrows():
  img_path = row['image']
  #img_path = os.path.join(img_folder, row['file'])
  img = tf.keras.utils.load_img(img_path, target_size=(224,224))
  img_array = tf.keras.utils.img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)

  img_pred_t = img_model.predict(img_array)
  img_pred_label = np.argmax(img_pred_t)
  img_pred_class = img_label_to_class.get(img_pred_label, 'None')
  img_predictions.append(img_pred_label)

  #plt.imshow(img)
  #plt.title(f'Predict : {img_pred_label}({img_pred_class}) \n {os.path.basename(img_path)}')
  #plt.axis('off')
  #plt.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━

In [ ]:
img_df5['pred'] = img_predictions()
img_df5

In [ ]:
img_df5.to_csv('01043620081_3.csv', index=False, encoding='utf-8')
img_df6 = pd.read_csv('01043620081_3.csv')
img_df6

In [ ]:
img_model.save('01043620081_3.h5')